In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
import keras as ks

First we load our dataset

In [14]:
CUR_DIR = Path().absolute()
DATA_DIR = CUR_DIR / "sentiment labelled sentences"

df_list = []

for data_file in DATA_DIR.glob("*.txt"):
    if data_file.name.lower() == "readme.txt": continue
    
    df = pd.read_csv(data_file, names=['sentence', 'label'], sep='\t')
    df['source'] = data_file.name.split("_")[0]  # Add another column filled with the source name
    df_list.append(df)

df = pd.concat(df_list)

Just to verify, we show the data

In [15]:
df

,sentence,label,source
0,So there is no way for me to plug it in here i...,0,amazon
1,"Good case, Excellent value.",1,amazon
2,Great for the jawbone.,1,amazon
3,Tied to charger for conversations lasting more...,0,amazon
4,The mic is great.,1,amazon
...,...,...,...
743,I just got bored watching Jessice Lange take h...,0,imdb
744,"Unfortunately, any virtue in this film's produ...",0,imdb
745,"In a word, it is embarrassing.",0,imdb
746,Exceptionally bad!,0,imdb


We now try to use the "Bag of Words" way to extract features - we generate a way to represent presence of works as a vector.

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

feature_extractor = CountVectorizer(min_df=0.0)

feature_extractor.fit(df["sentence"])

feature_extractor.vocabulary_  # Shows us global frequency of words

{'so': 4161,
 'there': 4545,
 'is': 2427,
 'no': 3043,
 'way': 4987,
 'for': 1829,
 'me': 2809,
 'to': 4609,
 'plug': 3387,
 'it': 2432,
 'in': 2314,
 'here': 2158,
 'the': 4531,
 'us': 4835,
 'unless': 4793,
 'go': 1982,
 'by': 647,
 'converter': 1013,
 'good': 1993,
 'case': 713,
 'excellent': 1603,
 'value': 4861,
 'great': 2023,
 'jawbone': 2449,
 'tied': 4589,
 'charger': 768,
 'conversations': 1012,
 'lasting': 2567,
 'more': 2931,
 'than': 4526,
 '45': 45,
 'minutes': 2890,
 'major': 2753,
 'problems': 3482,
 'mic': 2868,
 'have': 2113,
 'jiggle': 2464,
 'get': 1954,
 'line': 2648,
 'up': 4821,
 'right': 3777,
 'decent': 1171,
 'volume': 4928,
 'if': 2282,
 'you': 5138,
 'several': 4004,
 'dozen': 1373,
 'or': 3144,
 'hundred': 2259,
 'contacts': 987,
 'then': 4542,
 'imagine': 2294,
 'fun': 1901,
 'of': 3097,
 'sending': 3968,
 'each': 1440,
 'them': 4538,
 'one': 3124,
 'are': 273,
 'razr': 3611,
 'owner': 3200,
 'must': 2980,
 'this': 4558,
 'needless': 3010,
 'say': 3889,
 '

So each sentence now has a 0-1 valued vector representing which of the words are used in the review.

Now, we try to see which words predict a positive review. First, we need to split the data into training and testing sets

In [30]:
from sklearn.model_selection import train_test_split

sentences_train, sentences_test, y_train, y_test = train_test_split(
    df["sentence"], df["label"], test_size=0.25, random_state=42069,
)

f"{len(sentences_train) = }, {len(sentences_test) = }"

'len(sentences_train) = 2061, len(sentences_test) = 687'

Now we try to train a simple logistic regression model

In [34]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
x_train = feature_extractor.transform(sentences_train)

model.fit(x_train, y_train)

LogisticRegression()

We show the scores for train and test sets

In [35]:
f"{model.score(x_train, y_train) = }"

'model.score(x_train, y_train) = 0.9805919456574479'

In [36]:
x_test = feature_extractor.transform(sentences_test)

f"{model.score(x_test, y_test) = }"

'model.score(x_test, y_test) = 0.8195050946142649'

In [48]:
coefficients = model.coef_[0]

coefficient_scores = sorted([*zip(coefficients, feature_extractor.vocabulary_)])

# Worst 10 words indicating a bad review
coefficient_scores[:10]

[(-2.357836166505862, 'editing'),
 (-2.347710836285993, 'advertised'),
 (-1.9573985711753945, 'cgi'),
 (-1.9199586828726731, 'occasionally'),
 (-1.559459188563136, 'taxidermists'),
 (-1.5481850429561146, 'ipods'),
 (-1.456588117894457, 'revealing'),
 (-1.3736877123646227, 'captured'),
 (-1.3644534081090904, 'owed'),
 (-1.330398386093997, 'unrecognizable')]

In [49]:
# Best 10 words indicating a good review
coefficient_scores[-10:]

[(1.6372054802104714, 'ponyo'),
 (1.6795397628900823, 'kitchen'),
 (1.682489282544755, 'ask'),
 (1.6911432150381283, 'says'),
 (1.7671884554676516, 'tasteless'),
 (1.7674666075846632, 'sometimes'),
 (1.857047169866822, 'oysters'),
 (2.053246510373835, 'gone'),
 (2.075956679844248, 'anti'),
 (2.9158695967601296, 'milk')]